before start the testing, see Readme, be sure the IPbus, control Hub and python 3.8 is installed.
To properly set all paths run source setup.sh.

## Cell 1: KCU Connection Setup
This cell establishes connection to the KCU board and verifies basic communication.
- Sets KCU IP address to 192.168.0.10
- Enables control hub mode
- Performs firmware version check and loopback test

**Expected Output:** 
- "Successfully connected to KCU"
- Firmware version information
- "KCU Loopback test PASSED"


In [2]:
from tamalero.utils import get_kcu
import time
kcu_ip = "192.168.0.10" ## If your KCU ip is diff, modify it.

kcu = get_kcu(
    kcu_ip,
    control_hub=True,
    host='localhost',
    verbose=True
)
print("Successfully connected to KCU.")

##Check the KCU's status and firmware
time.sleep(0.1)
# kcu.status() # Prints LpGBT link statuses from KCU 
fw_ver = kcu.get_firmware_version(verbose=True) #
kcu.check_clock_frequencies(verbose=True) # Verifies KCU clock stability

##Perform a simple loopback register test to confirm communication
loopback_val = 0xABCD1234
kcu.write_node("LOOPBACK.LOOPBACK", loopback_val) #
read_val = kcu.read_node("LOOPBACK.LOOPBACK").value()
if read_val == loopback_val:
    print(f"KCU Loopback test PASSED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")
else:
    print(f"KCU Loopback test FAILED: Wrote 0x{loopback_val:X}, Read 0x{read_val:X}")

Using control hub on host=localhost, kcu_address=192.168.0.10
IPBus address: chtcp-2.0://localhost:10203?target=192.168.0.10:50001
Address table hash: d140783
KCU firmware version: 0.0.0
Successfully connected to KCU.
Firmware version: 2025/06/16 59:40:40 v0.0.0 sha=8538990
0x100B  r       FW_INFO.CLK125_FREQ                               0x07735940
0x1008  r       FW_INFO.CLK320_FREQ                               0x131C8852
0x1007  r       FW_INFO.CLK_40_FREQ                               0x02639109
0x1009  r       FW_INFO.REFCLK_FREQ                               0x131C8853
0x1018  r       FW_INFO.RXCLK0_FREQ                               0x131C8853
0x100E  r       FW_INFO.TXCLK0_FREQ                               0x131C8852
0x100F  r       FW_INFO.TXCLK1_FREQ                               0x131C8852
KCU Loopback test PASSED: Wrote 0xABCD1234, Read 0xABCD1234


This cell configures the ReadoutBoard object and establishes lpGBT communication.

Steps performed:
1. Initialize ReadoutBoard with modulev1 configuration
2. Detect and verify readout board version
3. Check lpGBT link status
4. Verify no FEC errors on DAQ link

Troubleshooting:
- If version detection fails, check lpGBT connection
- If FEC errors > 0, reset error counters and check links


In [10]:
from tamalero.ReadoutBoard import ReadoutBoard

rb = ReadoutBoard(
    rb=0,
    kcu=kcu,
    config="modulev1", 
    trigger=False,     
    verbose=False
)

print(f"Readout Board version detected: {rb.ver}")

0x1d7 readback value is 0x00
lpGBT version found False
1
0x1d7 readback value is 0x00
lpGBT version found False
2
0x1d7 readback value is 0xae
lpGBT version found True
 > lpGBT v2 detected
 > Running power up within LPGBT.configure()
Readout Board version detected: 2


- If FEC errors > 0, no worries, READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT was reset later
READOUT_BOARD_0.LPGBT.UPLINK_1.FEC_ERR_CNT is red due to our current setup only have one uplink which is uplink 0

In [4]:
# print("Checking DAQ LpGBT base configuration:")
# rb.DAQ_LPGBT.read_base_config() # check lpgbt register matched lpgbt.yaml or not

# # Read on-board temperatures
# print("\nReading temperatures:")
# rb.read_temp(verbose=True) #

rb.DAQ_LPGBT.read_adc(0)

rb.DAQ_LPGBT.read_adcs(check=True, strict_limits=True)

# print("\nReading DAQ LpGBT ADCs:")
# try:
#     rb.DAQ_LPGBT.read_adcs(check=True, strict_limits=True) #
# except ValueError as e:
#     print(f"LpGBT ADC check issue: {e}")

print(f"\nDAQ LpGBT Link Good: {rb.DAQ_LPGBT.link_status()}") 
# fec_errors = rb.get_FEC_error_count() # Reads KCU registers defined in READOUT_BOARD.xml
# print(f"FEC Errors: DAQ LpGBT = {fec_errors.get('DAQ', 'N/A')}")



┌────────────┬───────┬─────────────────┬───────────────────┬────────────────────┬──────────────────┬──────────┬──────────────────────────┐
│ Register   │   Pin │   Reading (raw) │   Reading (calib) │   Voltage (direct) │   Voltage (conv) │ Status   │ Comment                  │
├────────────┼───────┼─────────────────┼───────────────────┼────────────────────┼──────────────────┼──────────┼──────────────────────────┤
│ Vref4m     │     0 │             768 │          782      │           0.764418 │         0.764418 │ OK       │ Vref from ETROC B4 (A2)  │
│ VTEMP4     │     1 │             622 │          634.734  │           0.620464 │         0.620464 │ OK       │ VTEMP from ETROC B4 (A1) │
│ Vref1m     │     2 │             713 │          726.141  │           0.709815 │         0.709815 │ OK       │ Vref from ETROC B1 (B2)  │
│ VTEMP1     │     3 │             632 │          643.875  │           0.629399 │         0.629399 │ OK       │ VTEMP from ETROC B1 (B1) │
│ Vref3m     │     4 │     

Init ETROC2, our etroc2 i2c address is 0x63 and i2c channel is 1, modify it if yours diff

In [5]:
from tamalero.ETROC import ETROC
from tamalero.colors import green, red, yellow
from tamalero.LPGBT import LPGBT
import time

etroc_elinks_map = {0: [0, 4, 8, 12]} # should map your design
#etroc_elinks_map = {0: [12]}
print("Initializing ETROC object (I2C Addr: 0x63, Channel: 1)...")
try:
    my_etroc2 = ETROC(
        rb,
        master='lpgbt',
        i2c_adr=0x63,                 
        i2c_channel=1,
        elinks=etroc_elinks_map,
        strict=False,
        verbose=True 
    )

    if not my_etroc2.is_connected():
        raise ConnectionError("Failed to connect to ETROC2 via I2C.")

    print(green("SUCCESS: I2C communication with ETROC2 is established."))
    
    print("-" * 50)
    print("Verifying a key configuration register...")
    register_to_check = 'disScrambler'
    expected_value = 1
    
    read_back_value = my_etroc2.rd_reg(register_to_check)
    print(f"Reading back '{register_to_check}' register...")
    print(f"  - Value Expected: {expected_value}")
    print(f"  - Value Read Back: {read_back_value}")

    if read_back_value == expected_value:   ## check etroc register can be read correctly
        print(green("  - SUCCESS: Register read-back matches expected value."))
    else:
        print(red("  - FAILURE: Register read-back does NOT match expected value."))
        
    ## Debug and check status
    # print("-" * 50)
    # print("Next Step: Diagnosing the E-Link Lock Failure")
    # print("-" * 50)
    # register_to_test = 'BCIDoffset'
    # test_value = 0xAB 
    
    # print(f"  - Reading initial value of '{register_to_test}'...")
    # initial_value = my_etroc2.rd_reg(register_to_test)
    # print(f"  - Initial value: {hex(initial_value)}")
    
    # print(f"  - Writing test value {hex(test_value)} to '{register_to_test}'...")
    # my_etroc2.wr_reg(register_to_test, test_value)
    
    # print(f"  - Reading back value from '{register_to_test}'...")
    # read_back_value = my_etroc2.rd_reg(register_to_test)
    # print(f"  - Value read back: {hex(read_back_value)}")
    
    # if read_back_value == test_value:
    #     print(green("  - SUCCESS: Manual write/read test passed! I2C writes are working."))
    # else:
    #     print(red("  - FAILURE: Manual write/read test failed. I2C writes may not be working correctly."))
except Exception as e:
    print(red(f"An error occurred during register read-back: {e}"))

Initializing ETROC object (I2C Addr: 0x63, Channel: 1)...


elinks not locked, resetting PLL and FC modules
SUCCESS: I2C communication with ETROC2 is established.
--------------------------------------------------
Verifying a key configuration register...
Reading back 'disScrambler' register...
  - Value Expected: 1
  - Value Read Back: 1
  - SUCCESS: Register read-back matches expected value.


In [5]:
from tamalero.ETROC import ETROC
from tamalero.colors import green, red, yellow
from tamalero.LPGBT import LPGBT
import time

etroc_elinks_map = {0: [0, 4, 8, 12]} # should map your design
#etroc_elinks_map = {0: [12]}
print("Initializing ETROC object (I2C Addr: 0x63, Channel: 1)...")
try:
    etroc_chip1 = ETROC(
        rb,
        master='lpgbt',
        i2c_adr=0x63,                 
        i2c_channel=1,
        elinks=etroc_elinks_map,
        strict=False,
        verbose=True 
    )

    etroc_chip2 = ETROC(
        rb,
        master='lpgbt',
        i2c_adr=0x61,                 
        i2c_channel=1,
        elinks=etroc_elinks_map,
        strict=False,
        verbose=True 
    )
    etroc_chips = [etroc_chip1, etroc_chip2]
    chip_names = ['Chip1', "Chip2"]

    print("Both Etroc initialized")

    for i, (etroc, chip_name) in enumerate(zip(etroc_chips, chip_names)):
        print(f"\n=== {chip_name} Status ===")
        print(f"{chip_name} connected: {etroc.is_connected()}")

        print("Verifying a key configuration register...")
        register_to_check = 'disScrambler'
        expected_value = 1
        
        read_back_value = etroc.rd_reg(register_to_check)
        print(f"Reading back '{register_to_check}' register...")
        print(f"  - Value Expected: {expected_value}")
        print(f"  - Value Read Back: {read_back_value}")

        if read_back_value == expected_value:   ## check etroc register can be read correctly
            print(green("  - SUCCESS: Register read-back matches expected value."))
        else:
            print(red("  - FAILURE: Register read-back does NOT match expected value."))

        if hasattr(etroc, 'controllerState'):
            state = etroc.rd_reg("controllerState")
            print(f"Controller state: {state} (should be 11)")

        # Check Pll lock status
        Pll_state = etroc.rd_reg("pllUnlockCount")
        print(f"PLL Unlock Count: {Pll_state} (should be 0 or very few and would not increase)")

        # Check elink lock status
        print("\n=== Elink Status ===")
        elink_status = etroc.get_elink_status()
        print(f"Elink status: {elink_status}")


    for elink in [0, 4, 8, 12]: 
        locked = rb.etroc_locked(elink, slave=False)
        print(f"Elink {elink} locked: {locked}")
        
        if locked:
            packet_count = rb.read_packet_count(elink, slave=False)
            error_count = rb.read_error_count(elink, slave=False)
            filler_rate = rb.read_filler_rate(elink, slave=False)
            
            print(f"  Packets: {packet_count}")
            print(f"  Errors: {error_count}")
            print(f"  Filler rate: {filler_rate}")
            
    print('-' * 50)
    eprx_group_to_check = [0,1,2,3]
        
    try:
        for group in eprx_group_to_check:
            # Read the Read-Only status register
            dll_locked_status = rb.DAQ_LPGBT.rd_reg(f"LPGBT.RO.EPORTRX_RO.EPRX.EPRX{group}DLLLOCKED")
            if dll_locked_status == 1:
                print(green(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (Locked)"))
            else:
                print(red(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (NOT Locked)"))
    except Exception as e:
        print(red(f"dddCould not read LpGBT EPRX DLL status. Error: {e}"))

        
except Exception as e:
    print(red(f"An error occurred during register read-back: {e}"))

Initializing ETROC object (I2C Addr: 0x63, Channel: 1)...
elinks not locked, resetting PLL and FC modules
elinks not locked, resetting PLL and FC modules
Both Etroc initialized

=== Chip1 Status ===
Chip1 connected: 44
Verifying a key configuration register...
Reading back 'disScrambler' register...
  - Value Expected: 1
  - Value Read Back: 1
  - SUCCESS: Register read-back matches expected value.
Controller state: 11 (should be 11)
PLL Unlock Count: 4 (should be 0 or very few and would not increase)

=== Elink Status ===
Elink status: {0: [False, True, False, True]}

=== Chip2 Status ===
Chip2 connected: 44
Verifying a key configuration register...
Reading back 'disScrambler' register...
  - Value Expected: 1
  - Value Read Back: 1
  - SUCCESS: Register read-back matches expected value.
Controller state: 11 (should be 11)
PLL Unlock Count: 4 (should be 0 or very few and would not increase)

=== Elink Status ===
Elink status: {0: [False, True, False, True]}
Elink 0 locked: False
Elink

Just checking etroc status and see which elink is locked.
if elink is unlock, can't retrieve data from etroc

In [6]:
print(f"ETROC connected: {my_etroc2.is_connected()}")
print(f"ETROC version: {my_etroc2.get_ver()}")

# Check if ETROC is in a good state
if hasattr(my_etroc2, 'is_good'):
    print(f"ETROC is good: {my_etroc2.is_good()}")

# Check controller state
if hasattr(my_etroc2, 'controllerState'):
    state = my_etroc2.rd_reg("controllerState")
    print(f"Controller state: {state} (should be 11)")

# Check Pll lock status
Pll_state = my_etroc2.rd_reg("pllUnlockCount")
print(f"PLL Unlock Count: {Pll_state} (should be 0 or very few and would not increase)")

# Check elink lock status
print("\n=== Elink Status ===")
elink_status = my_etroc2.get_elink_status()
print(f"Elink status: {elink_status}")


for elink in [0, 4, 8, 12]: 
    locked = rb.etroc_locked(elink, slave=False)
    print(f"Elink {elink} locked: {locked}")
    
    if locked:
        packet_count = rb.read_packet_count(elink, slave=False)
        error_count = rb.read_error_count(elink, slave=False)
        filler_rate = rb.read_filler_rate(elink, slave=False)
        
        print(f"  Packets: {packet_count}")
        print(f"  Errors: {error_count}")
        print(f"  Filler rate: {filler_rate}")
        
print('-' * 50)
eprx_group_to_check = [0,1,2,3]
    
try:
    for group in eprx_group_to_check:
        # Read the Read-Only status register
        dll_locked_status = rb.DAQ_LPGBT.rd_reg(f"LPGBT.RO.EPORTRX_RO.EPRX.EPRX{group}DLLLOCKED")
        if dll_locked_status == 1:
            print(green(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (Locked)"))
        else:
            print(red(f"  - LpGBT EPRX Group {group} DLL Lock Status: {dll_locked_status} (NOT Locked)"))
except Exception as e:
    print(red(f"dddCould not read LpGBT EPRX DLL status. Error: {e}"))


ETROC connected: 44
ETROC version: 0-0-0
ETROC is good: True
Controller state: 11 (should be 11)
PLL Unlock Count: 4 (should be 0 or very few and would not increase)

=== Elink Status ===
Elink status: {0: [False, True, False, True]}
Elink 0 locked: False
Elink 4 locked: True
  Packets: 0
  Errors: 65535
  Filler rate: 16777215
Elink 8 locked: False
Elink 12 locked: True
  Packets: 0
  Errors: 65535
  Filler rate: 16777215
--------------------------------------------------
  - LpGBT EPRX Group 0 DLL Lock Status: 1 (Locked)
  - LpGBT EPRX Group 1 DLL Lock Status: 1 (Locked)
  - LpGBT EPRX Group 2 DLL Lock Status: 0 (NOT Locked)
  - LpGBT EPRX Group 3 DLL Lock Status: 0 (NOT Locked)


check uplink status and rese FEC errors if not 0

In [8]:
print("=== Check Uplink Status ===")

# Check uplink error counts
uplink0_errors = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.LPGBT.UPLINK_0.FEC_ERR_CNT").value()

print(f"Uplink 0 FEC errors: {uplink0_errors}")

# Check uplink ready status
uplink0_ready = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.LPGBT.UPLINK_0.READY").value()

print(f"Uplink 0 ready: {uplink0_ready}")

# Reset FEC error counters
print("\nResetting FEC error counters...")
rb.reset_FEC_error_count()

print("\nChecking elink 12 status again...")
rb.reset_data_error_count()
time.sleep(1)

for elinks in [0,4,8,12]: 
    locked = rb.etroc_locked(elinks, slave=False)
    packet_count = rb.read_packet_count(elinks, slave=False)
    error_count = rb.read_error_count(elinks, slave=False)
    filler_rate = rb.read_filler_rate(elinks, slave=False)

    print(f"Elink {elinks} - Locked: {locked}, Packets: {packet_count}, Errors: {error_count}, Filler: {filler_rate}")

    if filler_rate > 15000000 and error_count < 100:
        print(f"Elink {elinks} appears to be working!")
    else:
        print(f"Elink {elinks} still has issues")

=== Check Uplink Status ===
Uplink 0 FEC errors: 55
Uplink 0 ready: 1

Resetting FEC error counters...
Error counts before reset:
Address Perm.   Name                                              Value   
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT        0x00000037
0x2011  r       READOUT_BOARD_0.LPGBT.UPLINK_1.FEC_ERR_CNT        0x0000FFFF
Error counts after reset:
Address Perm.   Name                                              Value   
0x2001  r       READOUT_BOARD_0.LPGBT.UPLINK_0.FEC_ERR_CNT        0x00000000
0x2011  r       READOUT_BOARD_0.LPGBT.UPLINK_1.FEC_ERR_CNT        0x0000A2B7

Checking elink 12 status again...
Elink 0 - Locked: False, Packets: 0, Errors: 65535, Filler: 150
Elink 0 still has issues
Elink 4 - Locked: True, Packets: 0, Errors: 0, Filler: 16777215
Elink 4 appears to be working!
Elink 8 - Locked: False, Packets: 0, Errors: 65535, Filler: 0
Elink 8 still has issues
Elink 12 - Locked: True, Packets: 0, Errors: 0, Filler: 16777215
Elink 12 appear

just make sure lpGBT assignment is correct

In [13]:
print("=== Verify lpGBT Assignment ===")

print("Checking master vs slave elink locks...")

# Check master elinks (lpgbt=0)
master_locked = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.ETROC_LOCKED").value()
print(f"Master (lpgbt=0) elinks locked: {bin(master_locked)}")

# Check slave elinks (lpgbt=1) 
slave_locked = rb.kcu.read_node(f"READOUT_BOARD_{rb.rb}.ETROC_LOCKED_SLAVE").value()
print(f"Slave (lpgbt=1) elinks locked: {bin(slave_locked)}")

for elink in [0,4,8,12]:
    print(f"\nElink {elink}:")

    locked_master = rb.etroc_locked(elink, slave=False)
    print(f"  lpgbt=0: {locked_master}")
    
    locked_slave = rb.etroc_locked(elink, slave=True)
    print(f"  lpgbt=1: {locked_slave}")
    
    if locked_master:
        print(f"  → Elink {elink} is on lpgbt=0 (DAQ)")
    elif locked_slave:
        print(f"  → Elink {elink} is on lpgbt=1 (Trigger)")
    else:
        print(f"  → Elink {elink} not locked on either lpGBT")

print(f"\n2. Your ETROC elinks configuration:")
# print(f"elinks[0]: {my_etroc2.elinks.get(0, 'Not configured')}")
# print(f"elinks[1]: {my_etroc2.elinks.get(1, 'Not configured')}")

=== Verify lpGBT Assignment ===
Checking master vs slave elink locks...
Master (lpgbt=0) elinks locked: 0b1000000010000
Slave (lpgbt=1) elinks locked: 0b0

Elink 0:
  lpgbt=0: False
  lpgbt=1: False
  → Elink 0 not locked on either lpGBT

Elink 4:
  lpgbt=0: True
  lpgbt=1: False
  → Elink 4 is on lpgbt=0 (DAQ)

Elink 8:
  lpgbt=0: False
  lpgbt=1: False
  → Elink 8 not locked on either lpGBT

Elink 12:
  lpgbt=0: True
  lpgbt=1: False
  → Elink 12 is on lpgbt=0 (DAQ)

2. Your ETROC elinks configuration:


try to reproduce data from ETROC without charge injection
under workmode : 0(normal mode) data type should be only contain header and trailer only

In [23]:
from tamalero.FIFO import FIFO
from tamalero.DataFrame import DataFrame

print("\n=== Get data without charge injection ===")
df = DataFrame()
fifo = FIFO(rb)
my_etroc2.reset()

fifo.reset()
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()  
fifo.use_etroc_data()
time.sleep(1)

rb.DAQ_LPGBT.set_uplink_group_data_source("normal") 
my_etroc2.wr_reg("QInjEn", 0, broadcast=True)   ## ensure no charge injection
my_etroc2.wr_reg("workMode", 0, broadcast=True) ## workMode == 0 : normal mode else self-test
my_etroc2.wr_reg("disDataReadout", 0, broadcast=True) 
my_etroc2.wr_reg("singlePort", 1)   
my_etroc2.disable_QInj(broadcast= True) ## ensure no charge injection

zero_suppress_status = fifo.get_zero_suppress_status()
print(f"Zero suppression status: {hex(zero_suppress_status)}")

fifo.send_l1a(4096)  ## l1a value based on test_ETROC.py
header_count = 0
hit_counter = 0
filler_count = 0
trailer_count = 0

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    print(f"Date being read{occupancy}")
    if occupancy > 0:
        print(green("SUCCESS: Data is being generated!"))
        print(f"   FIFO returned {occupancy} data items")

        for j, word in enumerate(data):
            data_type, event_data = word[0], word[1]
            #print(f"  Event {j}: {data_type} -> {event_data}")

            if data_type == 'header':
                header_count += 1
            elif data_type == 'data':
                hit_counter += 1
                #test_results["parsed_hits"].append(event_data) # Add full hit data 
            elif data_type == 'filler':
                filler_count += 1
        #print(f"Total events processed: {}")
        print(f"Headers found: {header_count}")
        #print(f"Hits detected: {hit_counter}")
        print(f"Filler found {filler_count}")
    else:
        print(yellow("NO Data to analysis"))
except Exception as e:
    print(red(f"Read failed: {e}"))

fifo.reset()


# fifo.set_trigger_rate(1000)
# time.sleep(5)
# fifo.set_trigger_rate(0)


# occupancy = fifo.get_occupancy()
# lost_words = fifo.get_lost_word_count()

# print(f"Occupancy after continuous trigger: {occupancy}")
# print(f"Lost words: {lost_words}")
# time.sleep(1)


# if occupancy > 0 or lost_words > 0:
#     print("SUCCESS: Data is being generated!")
#     try:
#         # Use the simple read method
#         raw_data = fifo.read(dispatch=True)
#         print(f"✅ Successfully read {len(raw_data)} words!")
            
#         if len(raw_data) >= 2:
#             from tamalero.FIFO import merge_words
#             merged = merge_words(raw_data[:20]) 
                
#             print(f"Merged {len(merged)} events:")
#             for j, word in enumerate(merged[:10]):
#                 data_type, event_data = df.read(word)
                
#                 print(f"  Event {j}: {data_type} -> {event_data}")
            
#     except Exception as e:
#         print(f"Read failed: {e}")

    


=== Get data without charge injection ===
4
4
4
4
4
Zero suppression status: 0xfffffff
Date being read9337
SUCCESS: Data is being generated!
   FIFO returned 9337 data items
Headers found: 4678
Filler found 0


In [57]:
onchip_l1a_value = my_etroc2.rd_reg("onChipL1AConf")
print(f"onChipL1AConf  value: {onchip_l1a_value}")   ## should be 0


4
onChipL1AConf  value: 0


Reproduce ETROC data with charge injection, should see data type == data(hit)

In [ ]:
import numpy as np
import random
from datetime import datetime, timezone
import os
import pickle

print("\n=== Get data with charge injection ===")

results = []
init_occupancy = fifo.get_occupancy()
print(f"Befor rest fifo occupancy {init_occupancy}")
## System reset
my_etroc2.reset()
time.sleep(1)
fifo.reset()
time.sleep(1)
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()  
fifo.use_etroc_data()

after_occupancy = fifo.get_occupancy()
print(f"after rest fifo occupancy {after_occupancy}")

pixel_row = random.randint(0, 15)  
pixel_col = random.randint(0, 15)  
charge = 30
fifo_max = 8192 

print(f"FIFO maximum capacity: {fifo_max} words")
print(f"Pixel: Row {pixel_row}, Col {pixel_col}")
print(f"Charges: {charge} fC")

my_etroc2.wr_reg("singlePort", 1)   
#Disable all pixels first
my_etroc2.wr_reg("disDataReadout", 1, broadcast=True)
my_etroc2.wr_reg("QInjEn", 0, broadcast=True)
my_etroc2.wr_reg("enable_TDC", 0, broadcast=True)
time.sleep(1)

# Enable target pixel
my_etroc2.wr_reg("workMode", 0, row=pixel_row, col=pixel_col, broadcast=True)
my_etroc2.wr_reg("enable_TDC", 1, row=pixel_row, col=pixel_col, broadcast=False)
my_etroc2.wr_reg("disDataReadout", 0, row=pixel_row, col=pixel_col, broadcast=False)
time.sleep(1)

baseline, noise_width = my_etroc2.auto_threshold_scan(
    row=pixel_row, 
    col=pixel_col, 
    broadcast=False, 
    offset='auto',  
    use=True,      
    verbose=True
)

print(f"Calibrated threshold: Baseline={baseline:.1f}, NoiseWidth={noise_width:.1f}")

# Check what DAC value was set
applied_dac = my_etroc2.rd_reg('DAC', row=pixel_row, col=pixel_col)
print(f"Applied DAC value: {applied_dac}")
# The QSel register takes a value from 0-31 to represent 1-32 fC
my_etroc2.wr_reg('DAC', applied_dac, row=pixel_row, col=pixel_col, broadcast=False)
my_etroc2.wr_reg("QSel", charge - 1, row=pixel_row, col=pixel_col, broadcast=False)
my_etroc2.wr_reg("QInjEn", 1, row=pixel_row, col=pixel_col, broadcast=False)
time.sleep(1)

## Send L1A
L1A_cnt = 500
print(f"Sending L1A {L1A_cnt} ...")
fifo.send_QInj(count=L1A_cnt, delay=my_etroc2.QINJ_delay)
# l1a_rate = fifo.get_l1a_rate()
# print(f"L1A rate: {l1a_rate}")

time.sleep(1)
print(f"L1A Delay: {my_etroc2.QINJ_delay}")

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    print(f"   FIFO returned {occupancy} data items")

    output_dir = "Qinj_Output"
    os.makedirs(output_dir, exist_ok=True)

    if occupancy > 0:
        print(green("   SUCCESS: Data was generated!"))

        test_results = {
            "test_parameters": {
                "timestamp_utc": datetime.now(timezone.utc).isoformat(),
                "pixel_row": pixel_row,
                "pixel_col": pixel_col,
                "L1A_delay": my_etroc2.QINJ_delay,
                "charge_fC": charge,
                "l1a_sent": L1A_cnt
            },
            "run_summary": {},
            "analysis": {},
            "parsed_hits": [], # Storing all hit data dictionaries
            "raw_events": data
        }
        
        # Parse the data
        header_count = 0
        hit_counter = 0
        filler_count = 0
        trailer_count = 0
        toa_values = []
        tot_values = []
        cal_values = []

        print("\n--- Parsed Hit Data ---")
        for i, event in enumerate(data):  # printing first 10 to console
            #print(event)
            data_type, event_data = event[0], event[1]
            
            if data_type == 'header':
                header_count += 1
            elif data_type == 'filler':
                filler_count += 1
            elif data_type is None:
                continue
            elif data_type == 'trailer':
                trailer_count += 1
            elif data_type == 'data':
                hit_counter += 1
                test_results["parsed_hits"].append(event_data) # Add full hit data

                # Extract values
                toa = event_data.get('toa')
                tot = event_data.get('tot')
                cal = event_data.get('cal')
                row = event_data.get('row_id')
                col = event_data.get('col_id')
                
                if toa is not None: toa_values.append(toa)
                if tot is not None: tot_values.append(tot)
                if cal is not None: cal_values.append(cal)

                # Print the first 20 hits to the console
                if hit_counter <= 5: 
                    if row == pixel_row and col == pixel_col:
                        print(green(f"  Hit {hit_counter:>2}: Pixel(R{row}, C{col}) -> ToA: {toa:<4}, ToT: {tot:<4}, Cal: {cal:<4}"))
                    else:
                        print(yellow(f"  Hit {hit_counter:>2}: UNEXPECTED Pixel(R{row}, C{col}) -> ToA: {toa:<4}, ToT: {tot:<4}, Cal: {cal:<4}"))

        print(f"\n--- Summary ---")
        print(f"Total events processed: {len(data)}")
        print(f"Headers found: {header_count}")
        print(f"Hits detected: {hit_counter}")
        print(f"Filler found {filler_count}")
        print(f'trailer found {trailer_count}')
        

        # Populate and save the full results to file
        test_results["run_summary"]["total_events_processed"] = len(data)
        test_results["run_summary"]["headers_found"] = header_count
        test_results["run_summary"]["hits_detected"] = hit_counter

        if toa_values:
            avg_toa = np.mean(toa_values)
            test_results["analysis"]["avg_toa"] = avg_toa
            print(f"ToA values: {len(toa_values)} numeric values, avg = {avg_toa:.2f}")
        if tot_values:
            avg_tot = np.mean(tot_values)
            test_results["analysis"]["avg_tot"] = avg_tot
            print(f"ToT values: {len(tot_values)} numeric values, avg = {avg_tot:.2f}")
        if cal_values:
            avg_cal = np.mean(cal_values)
            test_results["analysis"]["avg_cal"] = avg_cal
            print(f"Cal values: {len(cal_values)} numeric values, avg = {avg_cal:.2f}")
            
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        ## Not using json due to standard json can't serialize Numpy data types
        pickle_filename = f"charge_injection_R{pixel_row}_C{pixel_col}_{timestamp}.pkl"
        pickle_filepath = os.path.join(output_dir, pickle_filename)
        
        try:
            with open(pickle_filepath, 'wb') as f:
                pickle.dump(test_results, f)
            print(green("Successfully saved pickle file."))
            print(green("Successfully saved results."))
        except Exception as e:
            print(f"Error saving file: {e}")
        
    else:
        print(red("FIFO returned no data. No hits detected."))
except Exception as e:
    print("Exception ", e)

# Cleanup
for _ in range(3):
    fifo.reset()
    rb.reset_data_error_count()
    my_etroc2.wr_reg("QInjEn", 0, broadcast=True)
    my_etroc2.wr_reg("QInjEn", 0, row=pixel_row, col=pixel_col)
    my_etroc2.wr_reg("disDataReadout", 1, broadcast=True)
    time.sleep(1)
print("Cleanup completed: Disabled QInj, re-enabled all pixels")


In [ ]:
#######multiple pixels ####

import numpy as np
import random
from datetime import datetime, timezone
import os
import pickle

print("\n=== Get data with charge injection ===")

results = []
init_occupancy = fifo.get_occupancy()
print(f"Befor rest fifo occupancy {init_occupancy}")
## System rese
my_etroc2.reset()
time.sleep(1)
fifo.reset()
time.sleep(1)
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()  
fifo.use_etroc_data()

after_occupancy = fifo.get_occupancy()
print(f"after rest fifo occupancy {after_occupancy}")

test_pixels = [(3,8),(2,11),(5,5)]
# pixel_row = random.randint(0, 15)  
# pixel_col = random.randint(0, 15)  
charge = 30
fifo_max = 8192 

print(f"FIFO maximum capacity: {fifo_max} words")
print(f"Pixel: Row {pixel_row}, Col {pixel_col}")
print(f"Charges: {charge} fC")

my_etroc2.wr_reg("singlePort", 1)   
#Disable all pixels first
my_etroc2.wr_reg("disDataReadout", 1, broadcast=True)
my_etroc2.wr_reg("QInjEn", 0, broadcast=True)
my_etroc2.wr_reg("enable_TDC", 0, broadcast=True)
time.sleep(1)



for pixel_row, pixel_col in test_pixels:
    # Enable target pixel
    my_etroc2.wr_reg("workMode", 0, row=pixel_row, col=pixel_col, broadcast=False)
    my_etroc2.wr_reg("enable_TDC", 1, row=pixel_row, col=pixel_col, broadcast=False)
    my_etroc2.wr_reg("disDataReadout", 0, row=pixel_row, col=pixel_col, broadcast=False)
    time.sleep(1)
    baseline, noise_width = my_etroc2.auto_threshold_scan(
        row=pixel_row, 
        col=pixel_col, 
        broadcast=False, 
        offset='auto',  
        use=True,      
        verbose=True
    )

    print(f"Calibrated threshold: Baseline={baseline:.1f}, NoiseWidth={noise_width:.1f}")

    # Check what DAC value was set
    applied_dac = my_etroc2.rd_reg('DAC', row=pixel_row, col=pixel_col)
    print(f"Applied DAC value: {applied_dac}")
    # The QSel register takes a value from 0-31 to represent 1-32 fC
    my_etroc2.wr_reg('DAC', applied_dac, row=pixel_row, col=pixel_col, broadcast=False)
    my_etroc2.wr_reg("QSel", charge - 1, row=pixel_row, col=pixel_col, broadcast=False)
    my_etroc2.wr_reg("QInjEn", 1, row=pixel_row, col=pixel_col, broadcast=False)
    time.sleep(1)

## Send L1A
L1A_cnt = 500
print(f"Sending L1A {L1A_cnt} ...")
fifo.send_QInj(count=L1A_cnt, delay=my_etroc2.QINJ_delay)

time.sleep(1)
print(f"L1A Delay: {my_etroc2.QINJ_delay}")

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    print(f"   FIFO returned {occupancy} data items")

    output_dir = "Qinj_Output"
    os.makedirs(output_dir, exist_ok=True)

    if occupancy > 0:
        print(green("   SUCCESS: Data was generated!"))

        test_results = {
            "test_parameters": {
                "timestamp_utc": datetime.now(timezone.utc).isoformat(),
                "test_pixels": test_pixels,
                "n_pixels": len(test_pixels),
                "L1A_delay": my_etroc2.QINJ_delay,
                "charge_fC": charge,
                "l1a_sent": L1A_cnt
            },
            "run_summary": {},
            "analysis": {},
            "pixel_results": {},
            "parsed_hits": [], # Storing all hit data dictionaries
            "raw_events": data
        }

        pixel_hit = {pixel: 0 for pixel in test_pixels}
        pixel_data = {pixel: {'toa':[], "tot":[], "cal":[]} for pixel in test_pixels}
        
        # Parse the data
        header_count = 0
        hit_counter = 0
        filler_count = 0
        trailer_count = 0

        print("\n--- Multi pixels_Parsed Hit Data ---")
        for i, event in enumerate(data):  # printing first 10 to console
            data_type, event_data = event[0], event[1]
            
            if data_type == 'header':
                header_count += 1
            elif data_type == 'filler':
                filler_count += 1
            elif data_type is None:
                continue
            elif data_type == 'trailer':
                trailer_count += 1
            elif data_type == 'data':
                hit_counter += 1
                test_results["parsed_hits"].append(event_data) # Add full hit data

                # Extract values
                toa = event_data.get('toa')
                tot = event_data.get('tot')
                cal = event_data.get('cal')
                row = event_data.get('row_id')
                col = event_data.get('col_id')
                
                if (row, col) in test_pixels:
                    pixel_hit[(row, col)] += 1
                    pixel_data[(row,col)]["toa"].append(toa)
                    pixel_data[(row,col)]["tot"].append(tot)
                    pixel_data[(row,col)]["cal"].append(cal)

                # Print the first 20 hits to the console
                    if pixel_hit[(row, col)] <= 10: 
                        print(green(f" Pixel({row},{col}) Hit {pixel_hit[(row, col)]} -> ToA: {toa}, ToT: {tot}, Cal: {cal}"))
                else:
                    print(yellow(f"  UNEXPECTED Pixel(R{row}, C{col}) -> ToA: {toa}, ToT: {tot}, Cal: {cal}"))

        print(f"\n--- Summary ---")
        print(f"Total events processed: {len(data)}")
        print(f"Headers found: {header_count}")
        print(f"Hits detected: {hit_counter}")
        print(f'trailer found {trailer_count}')
        print(f"Filler found {filler_count}")
        
        
        total_expected_hits = 0
        for pixel in test_pixels:
            pixel_row, pixel_col = pixel
            hits = pixel_hit[pixel]
            total_expected_hits += hits
            print(f"\n ---Pixel ({pixel_row}, {pixel_col}) Results ---")

            pixel_stats = {}
            if hits > 0:
                toa_values = pixel_data[pixel]["toa"]
                tot_values = pixel_data[pixel]["tot"]
                cal_values = pixel_data[pixel]["cal"]

                if toa_values:
                    pixel_stats["toa"] = {"mean" : np.mean(toa_values),}
                if tot_values:
                    pixel_stats["tot"] = {"mean" : np.mean(tot_values),}
                if cal_values:
                    pixel_stats["cal"] = {"mean" : np.mean(cal_values),}
            
            test_results["pixel_results"][f"pixel_{pixel_row}_{pixel_col}"] = {
                "position": pixel,
                "hits":hits,
                "stats": pixel_stats
            }
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        ## Not using json due to standard json can't serialize Numpy data types
        pickle_filename = f"multi_pixel_injection_{len(test_pixels)}_pixels_{timestamp}.pkl"
        pickle_filepath = os.path.join(output_dir, pickle_filename)
        
        try:
            with open(pickle_filepath, 'wb') as f:
                pickle.dump(test_results, f)
            print(green("Successfully saved pickle file."))
            print(green("Successfully saved results."))
        except Exception as e:
            print(f"Error saving file: {e}")
        
    else:
        print(red("FIFO returned no data. No hits detected."))
except Exception as e:
    print("Exception ", e)

# Cleanup
for _ in range(3):
    fifo.reset()
    rb.reset_data_error_count()
    my_etroc2.wr_reg("QInjEn", 0, broadcast=True)
    #my_etroc2.wr_reg("QInjEn", 0, row=pixel_row, col=pixel_col)
    my_etroc2.wr_reg("disDataReadout", 1, broadcast=True)
    time.sleep(1)
print("Cleanup completed: Disabled QInj, re-enabled all pixels")

In [ ]:
######multiple pixels ####
## Two Etroc 

import numpy as np
import random
from datetime import datetime, timezone
import os
import pickle
from tamalero.DataFrame import DataFrame
from tamalero.FIFO import FIFO

df = DataFrame()
fifo = FIFO(rb)
print("\n=== Get data with charge injection ===")

results = []

time.sleep(1)
fifo.reset()
time.sleep(1)
rb.reset_data_error_count()
rb.enable_etroc_readout()
rb.rerun_bitslip()  
fifo.use_etroc_data()

charge = 30
fifo_max = 8192 
test_pixels_chip1 = [(1,2)]
test_pixels_chip2 = [(2,5),(7,10)]

all_test_config = [
    (etroc_chip1, "Chip1", test_pixels_chip1),
    (etroc_chip2, "Chip2", test_pixels_chip2),
]


# for _ in range(3):
#     print(f"Elink4 invalid Fcc {etroc_chip2.get_invalidFCCount()}")

for etroc, chip_name, _ in all_test_config:
    print(f"Resetting {chip_name}")
    etroc.reset()

for etroc, chip_name, test_pixels in all_test_config:
    print(f"Configuring {chip_name} ...")
    etroc.wr_reg("singlePort", 1)   
    #Disable all pixels first
    etroc.wr_reg("disDataReadout", 1, broadcast=True)
    etroc.wr_reg("QInjEn", 0, broadcast=True)
    etroc.wr_reg("enable_TDC", 0, broadcast=True)
    etroc.wr_reg("workMode", 0, broadcast = True)
    time.sleep(1)


    for pixel_row, pixel_col in test_pixels:
        print(f"Configuring {chip_name} Pixel ({pixel_row}, {pixel_col})")

        # Enable target pixel
        etroc.wr_reg("workMode", 0, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("enable_TDC", 1, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("disDataReadout", 0, row=pixel_row, col=pixel_col, broadcast=False)
        time.sleep(1)
        print(f" {chip_name} Invalid FCC {etroc.get_invalidFCCount()} ")
        baseline, noise_width = etroc.auto_threshold_scan(
            row=pixel_row, 
            col=pixel_col, 
            broadcast=False, 
            offset='auto',  
            use=True,      
            verbose=True
        )

        print(f"Calibrated threshold: Baseline={baseline:.1f}, NoiseWidth={noise_width:.1f}")

        # Check what DAC value was set
        applied_dac = etroc.rd_reg('DAC', row=pixel_row, col=pixel_col)
        print(f"Applied DAC value: {applied_dac}")
        # The QSel register takes a value from 0-31 to represent 1-32 fC
        etroc.wr_reg('DAC', applied_dac, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("QSel", charge - 1, row=pixel_row, col=pixel_col, broadcast=False)
        etroc.wr_reg("QInjEn", 1, row=pixel_row, col=pixel_col, broadcast=False)
        time.sleep(1)

## Send L1A
L1A_cnt = 500
print(f"Sending L1A {L1A_cnt} ...")
fifo.send_QInj(count=L1A_cnt, delay=etroc.QINJ_delay)
#fifo.send_QInj(count=L1A_cnt, delay=484)
time.sleep(2)
print(f"L1A Delay: {etroc.QINJ_delay}")

try:
    data = fifo.pretty_read(df)
    occupancy = len(data)
    print(f"   FIFO returned {occupancy} data items")

    output_dir = "Qinj_Output"
    os.makedirs(output_dir, exist_ok=True)

    if occupancy > 0:
        print(green("   SUCCESS: Data was generated!"))
        all_test_pixels = test_pixels_chip1 + test_pixels_chip2

        test_results = {
            "test_parameters": {
                "timestamp_utc": datetime.now(timezone.utc).isoformat(),
                "all_test_pixels": all_test_pixels,
                "chip1_pixels": test_pixels_chip1,
                "chip2_pixels": test_pixels_chip2,
                "n_pixels": len(all_test_pixels),
                "L1A_delay": 504,
                "charge_fC": charge,
                "l1a_sent": L1A_cnt
            },
            "run_summary": {},
            "analysis": {},
            "chip_results": {},
            "parsed_hits": [], # Storing all hit data dictionaries
            "raw_events": data
        }
        chip_hits = {"Chip1": {}, "Chip2": {}}
        chip_data = {"Chip1": {}, "Chip2": {}}
        for etroc, chip_name, test_pixels in all_test_config:
            for pixel in test_pixels:
                chip_hits[chip_name][pixel]= 0
                chip_data[chip_name][pixel]= {'toa':[], "tot":[], "cal":[]} 
        
        # Parse the data
        header_count = 0
        hit_counter = 0
        filler_count = 0
        trailer_count = 0

        print("\n--- Multi pixels_Parsed Hit Data ---")
        for i, event in enumerate(data):  # printing first 10 to console
            if event is None or len(event) < 2:
                continue

            data_type, event_data = event[0], event[1]
            if data_type == 'header':
                header_count += 1
            elif data_type == 'filler':
                filler_count += 1
            elif data_type == 'trailer':
                trailer_count += 1
            elif data_type == 'data':
                hit_counter += 1
                test_results["parsed_hits"].append(event_data) # Add full hit data
                # Extract values
                toa = event_data.get('toa')
                tot = event_data.get('tot')
                cal = event_data.get('cal')
                row = event_data.get('row_id')
                col = event_data.get('col_id')
                
                for etroc, chip_name, test_pixels in all_test_config:
                    if (row, col) in test_pixels:
                        chip_hits[chip_name][(row, col)] += 1
                        chip_data[chip_name][(row, col)]["toa"].append(toa)
                        chip_data[chip_name][(row, col)]["tot"].append(tot)
                        chip_data[chip_name][(row, col)]["cal"].append(cal)

                    # Print the first 20 hits to the console
                        if chip_hits[chip_name][(row, col)] <= 10: 
                            print(green(f" {chip_name} Pixel({row},{col}) Hit {chip_hits[chip_name][(row, col)]} -> ToA: {toa}, ToT: {tot}, Cal: {cal}"))

        print(f"\n--- Summary ---")
        print(f"Total events processed: {len(data)}")
        print(f"Headers found: {header_count}")
        print(f"Hits detected: {hit_counter}")
        print(f'trailer found {trailer_count}')
        print(f"Filler found {filler_count}")
        
        total_expected_hits = 0
        for etroc, chip_name, test_pixels in all_test_config:
            for pixel in test_pixels:
                pixel_row, pixel_col = pixel
                hits = chip_hits[chip_name][pixel]
                total_expected_hits += hits
                print(f"\n ---Pixel ({pixel_row}, {pixel_col}) Results ---")

                pixel_stats = {}
                if hits > 0:
                    toa_values = chip_data[chip_name][pixel]["toa"]
                    tot_values = chip_data[chip_name][pixel]["tot"]
                    cal_values = chip_data[chip_name][pixel]["cal"]

                    if toa_values:
                        pixel_stats["toa"] = {"mean" : np.mean(toa_values),}
                    if tot_values:
                        pixel_stats["tot"] = {"mean" : np.mean(tot_values),}
                    if cal_values:
                        pixel_stats["cal"] = {"mean" : np.mean(cal_values),}
                
                test_results["chip_results"][f"{chip_name}_pixel_{pixel_row}_{pixel_col}"] = {
                    "chip": chip_name,
                    "position": pixel,
                    "hits":hits,
                    "stats": pixel_stats
                }
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        ## Not using json due to standard json can't serialize Numpy data types
        pickle_filename = f"multi_board_injection_{len(all_test_config)}_pixels_{timestamp}.pkl"
        pickle_filepath = os.path.join(output_dir, pickle_filename)
        
        try:
            with open(pickle_filepath, 'wb') as f:
                pickle.dump(test_results, f)
            print(green("Successfully saved pickle file."))
            print(green("Successfully saved results."))
        except Exception as e:
            print(f"Error saving file: {e}")
        
    else:
        print(red("FIFO returned no data. No hits detected."))
except Exception as e:
    print("Exception ", e)

# print("After Data acq, check invalid Fcc")
# for _ in range(3):
#     print(f"Elink 4 invalid Fcc {etroc_chip2.get_invalidFCCount()}")
# Cleanup
for etroc, chip_name, _ in all_test_config:
    print(f"Cleaning up {chip_name}...")
    for _ in range(3):
        fifo.reset()
        rb.reset_data_error_count()
        etroc.wr_reg("QInjEn", 0, broadcast=True)
        #my_etroc2.wr_reg("QInjEn", 0, row=pixel_row, col=pixel_col)
        etroc.wr_reg("disDataReadout", 1, broadcast=True)
        time.sleep(1)
print("Cleanup completed: Disabled QInj, re-enabled all pixels")


=== Get data with charge injection ===
Elink4 invalid Fcc 728
Elink4 invalid Fcc 728
Elink4 invalid Fcc 728
Resetting Chip1
Resetting Chip2
Configuring Chip1 ...
Configuring Chip1 Pixel (1, 2)
 Chip1 Invalid FCC 1373 
Calibrated threshold: Baseline=407.0, NoiseWidth=4.0
Applied DAC value: 411
Configuring Chip2 ...
Configuring Chip2 Pixel (2, 5)
 Chip2 Invalid FCC 728 
Calibrated threshold: Baseline=573.0, NoiseWidth=4.0
Applied DAC value: 577
Configuring Chip2 Pixel (7, 10)
 Chip2 Invalid FCC 728 
Calibrated threshold: Baseline=569.0, NoiseWidth=4.0
Applied DAC value: 573
Sending L1A 500 ...
L1A Delay: 504
Data read failed
   FIFO returned 3500 data items
   SUCCESS: Data was generated!

--- Multi pixels_Parsed Hit Data ---
 Chip1 Pixel(1,2) Hit 1 -> ToA: 264, ToT: 73, Cal: 168
 Chip2 Pixel(7,10) Hit 1 -> ToA: 490, ToT: 17, Cal: 158
 Chip2 Pixel(2,5) Hit 1 -> ToA: 253, ToT: 66, Cal: 160
 Chip1 Pixel(1,2) Hit 2 -> ToA: 263, ToT: 73, Cal: 168
 Chip2 Pixel(7,10) Hit 2 -> ToA: 474, ToT: 1

In [11]:
import pickle

with open("Qinj_Output/multi_board_injection_2_pixels_2025-06-18_16-37-55.pkl", 'rb') as f:
    data = pickle.load(f)

print("=== ALL PARAMETERS ===\n")

for main_key in ['test_parameters', 'run_summary', 'analysis', 'parsed_hits', 'raw_events', 'chip_results']:
    print(f" {main_key.upper().replace('_', ' ')}:")
    
    if main_key in data:
        if main_key == 'parsed_hits':
            print(f"   Count: {len(data[main_key])}")
            for i, hit in enumerate(data[main_key][:10]):
                print(f"   Hit {i+1}: {hit}")
                
        elif main_key == 'raw_events':
            print(f"   Count: {len(data[main_key])}")
            # Show first 10 as examples
            for i, event in enumerate(data[main_key][:40]):
                print(f"   Event {i+1}: {event}")
            if len(data[main_key]) > 10:
                print(f"   ... and {len(data[main_key])-40} more events")
        
        # check if data is a dictionary
        elif isinstance(data[main_key], dict):
            for key, value in data[main_key].items():
                print(f"   {key}: {value}")
        else:
            print(f"   {data[main_key]}")
    else:
        print("   Not found")
    
    print()  # Empty line between sections

=== ALL PARAMETERS ===

 TEST PARAMETERS:
   timestamp_utc: 2025-06-18T21:37:55.285032+00:00
   all_test_pixels: [(1, 2), (2, 5), (7, 10)]
   chip1_pixels: [(1, 2)]
   chip2_pixels: [(2, 5), (7, 10)]
   n_pixels: 3
   L1A_delay: 504
   charge_fC: 30
   l1a_sent: 500

 RUN SUMMARY:

 ANALYSIS:

 PARSED HITS:
   Count: 1500
   Hit 1: {'ea': 0, 'col_id': 10, 'row_id': 7, 'toa': 256, 'cal': 159, 'tot': 66, 'elink': 4, 'full': 0, 'any_full': 0, 'global_full': 0, 'raw': '0x94e801089f', 'raw_full': '0x494e801089f', 'meta': '0x4'}
   Hit 2: {'ea': 0, 'col_id': 5, 'row_id': 2, 'toa': 254, 'cal': 162, 'tot': 66, 'elink': 4, 'full': 0, 'any_full': 0, 'global_full': 0, 'raw': '0x8a47f108a2', 'raw_full': '0x48a47f108a2', 'meta': '0x4'}
   Hit 3: {'ea': 0, 'col_id': 2, 'row_id': 1, 'toa': 264, 'cal': 169, 'tot': 74, 'elink': 12, 'full': 0, 'any_full': 0, 'global_full': 0, 'raw': '0x84284128a9', 'raw_full': '0xc84284128a9', 'meta': '0xc'}
   Hit 4: {'ea': 0, 'col_id': 10, 'row_id': 7, 'toa': 256, 'ca

In [9]:
etroc_chip2.wr_reg("FC_InvData", 1)
for _ in range(3):
    print(etroc_chip2.get_invalidFCCount())

4
4
4
2430
4
4
2430
4
4
2430
